# 1. Linear Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import dataset

In [7]:
data = pd.read_excel("./data/boston.xls")
print(data.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM        AGE     DIS  RAD  TAX  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.199997  4.0900    1  296   
1  0.02731   0.0   7.07     0  0.469  6.421  78.900002  4.9671    2  242   
2  0.02729   0.0   7.07     0  0.469  7.185  61.099998  4.9671    2  242   
3  0.03237   0.0   2.18     0  0.458  6.998  45.799999  6.0622    3  222   
4  0.06905   0.0   2.18     0  0.458  7.147  54.200001  6.0622    3  222   

          PT           B  LSTAT         MV  
0  15.300000  396.899994   4.98  24.000000  
1  17.799999  396.899994   9.14  21.600000  
2  17.799999  392.829987   4.03  34.700001  
3  18.700001  394.630005   2.94  33.400002  
4  18.700001  396.899994   5.33  36.200001  


### Preprocessing: Feature Normalisation
Extract features and normalise them

In [8]:
def feature_normalisation(X):
    mu = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mu) / std

In [9]:
data = data.values
dataset_size = len(data)
np.random.shuffle(data)

X = data[:, :-1]
y = data[:, -1]
X = feature_normalisation(X)


## Gradient Descent

In [10]:
def gradient_descent(X, y, alpha=0.1, time=100, lam=None, reg='ridge'):
    # Adding intercept
    X = np.hstack((np.ones((len(X), 1)), X))
    n = len(X)
    theta = np.zeros(len(X[0]))
    rmse = np.zeros(time)
    for i in range(time):
        pred = np.dot(X, theta)
        error = np.subtract(pred, y)
        derivative = (alpha / n) * np.dot(X.T, error)
        if lam is not None:
            if reg == 'ridge':
                derivative -= (lam / n) * theta
            elif reg == 'lasso':
                derivative -= (lam / n) * theta

        theta = theta - derivative

        new_pred = np.dot(X, theta)
        new_error = np.subtract(new_pred, y)
        rmse[i] = np.sqrt((1. / n) * np.dot(new_error.T, new_error))
    return theta, rmse


In [11]:
alpha = 0.01
time = 100
k = 5
train_size = int(dataset_size / k)
theta = [None] * k

plt.figure()

for i in range(k):
    X_fold = X[i * train_size:(i + 1) * train_size, :]
    y_fold = y[i * train_size:(i + 1) * train_size]
    theta[i], rmse = gradient_descent(X_fold, y_fold, time=time)

    print(rmse[-1])

    plt.plot(rmse, label='Fold: %s' % (i + 1))

plt.legend()
plt.xlabel('Time')
plt.ylabel('RMSE')
plt.savefig("./q1_(" + str(1) + ").png")
# plt.show()
plt.close()

4.532786513004417
4.9667092975597145
4.624588268730105
4.483358012512484
3.6173584470680376


In [12]:
#  RMSE
def RMSE(X, y, theta):
    X = np.hstack((np.ones((len(X), 1)), X))
    error = np.subtract(np.dot(X, theta), y)
    return np.sqrt((1. / len(X)) * np.dot(error.T, error))


In [13]:
for j in range(k):
    X_fold = X[j * train_size:(j + 1) * train_size, :]
    y_fold = y[j * train_size:(j + 1) * train_size]
    print("Fold: ", j + 1, RMSE(X_fold, y_fold, theta[j]))


Fold:  1 4.532786513004417
Fold:  2 4.9667092975597145
Fold:  3 4.624588268730105
Fold:  4 4.483358012512484
Fold:  5 3.6173584470680376


## Regularization

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

In [15]:
parameters = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
reg = Ridge(alpha=1.0)
reg.fit(X, y)
y_pred = reg.predict(X)
print(np.sqrt((1 / len(X)) * np.dot((y_pred - y),(y_pred - y))))


4.679301462189175


In [16]:
l = 1.0  # Regularisation Parameter


In [17]:
alpha = 0.01
time = 100
k = 5
train_size = int(dataset_size / k)
theta = [None] * k

plt.figure()

for i in range(k):
    X_fold = X[i * train_size:(i + 1) * train_size, :]
    y_fold = y[i * train_size:(i + 1) * train_size]
    theta[i], rmse = gradient_descent(X_fold, y_fold, time=time, lam=l, reg='lasso')

    print(rmse[-1])
    
_, e = gradient_descent(X, y, time=time, lam=l)
print(e[-1])

5.264471254743731
5.599842136673281
5.282854319540708
5.132626553171499
4.514406213277949
4.722943131032555


<Figure size 432x288 with 0 Axes>

In [18]:
for j in range(k):
    X_fold = X[j * train_size:(j + 1) * train_size, :]
    y_fold = y[j * train_size:(j + 1) * train_size]
    print("Fold: ", j + 1, RMSE(X_fold, y_fold, theta[j]))


Fold:  1 5.264471254743731
Fold:  2 5.599842136673281
Fold:  3 5.282854319540708
Fold:  4 5.132626553171499
Fold:  5 4.514406213277949
